# Class 5: APIs

## APIs

- They are very powerful
- Can be used from any programming language
- Not expecting you to use them in your Final Project

## APIs, conceptually

![Diagram showing how online payments work: Expedia talks to Delta, Delta talks to Stripe, Stripe talks to Visa, and Visa talks to Chase](extras/img/apis_conceptually/payments.png)

![Diagram showing how notifications flow through systems](extras/img/apis_conceptually/notifications.png)

![Diagram showing relationship between human languages, programming languages, and APIs](extras/img/apis_conceptually/languages.png)

interactions between systems ↔️

## Ways to get data

Method | How it happens | Pros | Cons
--- | :--- | :--- | :---
**Bulk** | Download, someone hands you a flash drive, etc. | Fast, one-time transfer | Can be large; data gets out of date easily
**APIs** | If organization makes one available | Usually allows some filtering; can always pull latest-and-greatest | Requires network connection for every call; higher barrier to entry (reading documentation); subject to availability and performance of API
**Scraping** | Data only available through a web site, PDF, or doc | You can turn anything into data | Tedious; fragile

## Scraping

Common tools:

- [Beautiful Soup package](https://realpython.com/beautiful-soup-web-scraper-python/)
- [pandas' `read_html()`](https://pandas.pydata.org/docs/user_guide/io.html#html)

_Please pray to the Demo Gods that these all work and there's no profanity_

Pull table from [Wikipedia's list of countries by area](https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_area#Countries_and_dependencies_by_area):

In [1]:
import pandas as pd

tables = pd.read_html(
    "https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_area",
    match="Country / dependency",
)
countries = tables[0]
countries

,Unnamed: 0,Country / dependency,Total in km2 (mi2),Land in km2 (mi2),Water in km2 (mi2),% water,Unnamed: 6
0,NaN,Earth,"510,072,000 (196,940,000)","148,940,000 (57,506,000)","361,132,000 (139,434,000)",70.8,NaN
1,1,Russia,"17,098,246 (6,601,667)","16,378,410 (6,323,737)","719,836 (277,930)",4.2,[b]
2,NaN,Antarctica,"14,200,000 (5,480,000)","14,200,000 (5,480,000)",0,0.0,[c]
3,2,Canada,"9,984,670 (3,855,100)","9,093,507 (3,511,021)","891,163 (344,080)",8.9,[d]
4,3 or 4 [e],China,"9,596,961 (3,705,406)","9,326,410 (3,600,950)","270,550 (104,460)",2.8,[f][1]
...,...,...,...,...,...,...,...
260,NaN,Ashmore and Cartier Islands (Australia),5.0 (1.9),5.0 (1.9),0,0.0,NaN
261,NaN,Spratly Islands (disputed),2.0 (0.77),2.0 (0.77),0,0.0,NaN
262,NaN,Coral Sea Islands (Australia),2.0 (0.77),2.0 (0.77),0,0.0,NaN
263,194,Monaco,2.0 (0.77),2.0 (0.77),0,0.0,[dp][36]


### Data is only available if it's available

## API calls in the wild

1. Go to [Candidates page on fec.gov](https://www.fec.gov/data/candidates/?has_raised_funds=true&is_active_candidate=true).
1. Right click and `Inspect`.
   - [More info about opening Developer Tools in various browsers.](https://developer.mozilla.org/en-US/docs/Learn/Common_questions/What_are_browser_developer_tools#how_to_open_the_devtools_in_your_browser)
1. Go to the `Network` tab and reload.
1. Filter to `XHR`.
1. Click the API call.

We only see this because the tables on [fec.gov](https://fec.gov) are [rendered client-side](https://www.solutelabs.com/blog/client-side-vs-server-side-rendering-what-to-choose-when) using their JSON API. That won't be the case for all tables on all sites.

### Parts of a URL

![URL structure](https://developer.mozilla.org/en-US/docs/Learn/Common_questions/Web_mechanics/What_is_a_URL/mdn-url-all.png)

[source](https://developer.mozilla.org/en-US/docs/Learn/Common_questions/What_is_a_URL#basics_anatomy_of_a_url)

For APIs:

- Often split into "base URL" + "endpoint"
- Endpoints are like function names: they represent the information you are retrieving or thing you are trying to do
- Parameters are like function arguments:
   - They allow options to be specified
   - Some are required, some are optional
   - They will differ from one endpoint/function to another
- Anchors won't be used

### API documentation

[FEC API](https://api.open.fec.gov/developers/)

### Try it out

1. Visit https://www.fec.gov/data/candidates/
1. [Open Developer Tools](https://developer.mozilla.org/en-US/docs/Learn/Common_questions/What_are_browser_developer_tools#how_to_open_the_devtools_in_your_browser).
1. Reload the page.
1. In the Network tab's request list:
   1. Filter to Fetch/XHR/AJAX (terminology will differ by browser)
   1. Right-click the API call.
1. Click `Open in New Tab`.
1. Replace the `api_key` value with `DEMO_KEY`. The URL should therefore contain `api_key=DEMO_KEY`.

You should see a big wall of JSON data.

## API calls from Python

Usually one of two ways:

- A software development kit (SDK) like [sodapy](https://pypi.org/project/sodapy/)
   - Abstracts the details away
   - Not available for all APIs
   - May have limitations
- [The `requests` package](https://docs.python-requests.org/) (nothing to do with 311 requests)

Get Jimmy McMillan's latest candidacy information:

In [2]:
import requests

params = {
    "api_key": "DEMO_KEY",
    "q": "Jimmy McMillan",
    "sort": "-first_file_date",
}
response = requests.get("https://api.open.fec.gov/v1/candidates/", params=params)
data = response.json()
data

{'api_version': '1.0',
 'pagination': {'count': 2, 'page': 1, 'pages': 1, 'per_page': 20},
 'results': [{'active_through': 2016,
   'candidate_id': 'P60016805',
   'candidate_inactive': False,
   'candidate_status': 'N',
   'cycles': [2016, 2018],
   'district': '00',
   'district_number': 0,
   'election_districts': ['00'],
   'election_years': [2016],
   'federal_funds_flag': False,
   'first_file_date': '2015-10-13',
   'has_raised_funds': False,
   'inactive_election_years': None,
   'incumbent_challenge': 'O',
   'incumbent_challenge_full': 'Open seat',
   'last_f2_date': '2015-10-13',
   'last_file_date': '2015-10-13',
   'load_date': '2018-02-17T09:16:20',
   'name': 'MCMILLAN, JIMMY "RENT IS TOO DAMN HIGH',
   'office': 'P',
   'office_full': 'President',
   'party': 'REP',
   'party_full': 'REPUBLICAN PARTY',
   'state': 'US'},
  {'active_through': 2012,
   'candidate_id': 'P60003290',
   'candidate_inactive': False,
   'candidate_status': 'N',
   'cycles': [1996, 1998, 2012, 

### Retrieving nested data

In [3]:
data["results"][0]["name"]

'MCMILLAN, JIMMY "RENT IS TOO DAMN HIGH'

### In-class exercise

[Geocode](https://en.wikipedia.org/wiki/Address_geocoding) an address from Python using the [Nominatim API](https://nominatim.org/release-docs/develop/api/Search/). Print out the latitude and longitude. Any address is fine:

- Your own
- This building
- etc.

**Hint:** Try getting the API call working in your browser URL bar before calling it in Python.

### Reading into a DataFrame

In [4]:
params = {
    "api_key": "DEMO_KEY",
    "has_raised_funds": "true",
}
response = requests.get("https://api.open.fec.gov/v1/candidates/", params=params)
data = response.json()

pd.DataFrame(data["results"])

,active_through,candidate_id,candidate_inactive,candidate_status,cycles,district,district_number,election_districts,election_years,federal_funds_flag,...,incumbent_challenge_full,last_f2_date,last_file_date,load_date,name,office,office_full,party,party_full,state
0,2022,H2CO07170,False,P,"[2022, 2024]",07,7,[07],[2022],False,...,Open seat,2022-08-10,2022-08-10,2023-03-09T10:16:03,"AADLAND, ERIK",H,House,REP,REPUBLICAN PARTY,CO
1,2022,H2UT03280,False,C,[2022],03,3,[03],[2022],False,...,Challenger,2022-03-21,2022-03-21,2022-04-13T21:10:09,"AALDERS, TIM",H,House,REP,REPUBLICAN PARTY,UT
2,2018,S2UT00229,False,P,"[2012, 2014, 2016, 2018, 2020]",00,0,"[00, 00]","[2012, 2018]",False,...,Open seat,2018-04-23,2018-04-23,2019-03-27T16:02:41,"AALDERS, TIMOTHY NOEL",S,Senate,CON,CONSTITUTION PARTY,UT
3,2020,H0TX22260,False,C,[2020],22,22,[22],[2020],False,...,Open seat,2019-10-17,2019-10-17,2020-03-18T21:13:37,"AALOORI, BANGAR REDDY",H,House,REP,REPUBLICAN PARTY,TX
4,1978,H6PA16106,False,P,"[1976, 1978, 1980]",16,16,"[16, 16]","[1976, 1978]",False,...,None,1978-07-05,1978-07-05,2002-03-30T00:00:00,"AAMODT, NORMAN O.",H,House,REP,REPUBLICAN PARTY,PA
5,2012,H2CA01110,False,P,"[2012, 2014, 2016]",01,1,[01],[2012],False,...,Challenger,2012-02-22,2012-02-22,2013-04-26T09:04:30,"AANESTAD, SAMUEL",H,House,REP,REPUBLICAN PARTY,CA
6,2018,H8CO06237,False,C,[2018],06,6,[06],[2018],False,...,Challenger,2017-04-26,2017-04-26,2017-08-01T20:57:28,"AARESTAD, DAVID",H,House,DEM,DEMOCRATIC PARTY,CO
7,2008,P80002926,False,N,"[2006, 2008, 2010, 2012, 2014, 2016]",00,0,[00],[2008],False,...,Open seat,2007-03-13,2007-03-13,2016-11-17T06:10:48,"AARON, LAURA DAVIS",P,President,DEM,DEMOCRATIC PARTY,US
8,2024,H4OR05312,False,C,[2024],05,5,[05],[2024],False,...,Challenger,2023-07-24,2023-07-24,2023-10-16T21:03:02,"AASEN, ANDREW J",H,House,NON,NON-PARTY,OR
9,2024,H2CA30291,False,N,"[2022, 2024]",32,32,"[32, 32]","[2022, 2024]",False,...,Challenger,2022-07-15,2022-07-15,2023-01-12T22:24:01,"AAZAMI, SHERVIN",H,House,DEM,DEMOCRATIC PARTY,CA


## Back to 311 data

From [NYC Open Data Portal dataset page](https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9/data), click `Export` -> `SODA API` -> `API Docs`.

### Most open data sites have APIs

Often built on platforms that provide them, e.g.

- [NYC Open Data Portal](https://opendata.cityofnewyork.us/) built on [Socrata](https://dev.socrata.com/)
- [data.gov built on CKAN](https://www.data.gov/developers/apis)

### Example: 311 requests from the last week

In [5]:
from datetime import datetime, timedelta

now = datetime.utcnow()
now

datetime.datetime(2023, 12, 6, 8, 18, 46, 607845)

In [6]:
start = now - timedelta(weeks=1)
start

datetime.datetime(2023, 11, 29, 8, 18, 46, 607845)

In [7]:
start.isoformat()

'2023-11-29T08:18:46.607845'

Using the [Socrata query language (SoQL)](https://dev.socrata.com/docs/queries/):

In [8]:
data_id = "erm2-nwe9"
params = {
    "$where": f"created_date between '{start.isoformat()}' and '{now.isoformat()}'",
    # keeping the result size small for the purposes of this notebook
    "$limit": 10
}

url = f"https://data.cityofnewyork.us/resource/{data_id}.json"
response = requests.get(url, params=params)
data = response.json()

data

[{'unique_key': '59632748',
  'created_date': '2023-12-05T12:00:00.000',
  'agency': 'DSNY',
  'agency_name': 'Department of Sanitation',
  'complaint_type': 'Derelict Vehicles',
  'descriptor': 'Derelict Vehicles',
  'location_type': 'Street',
  'incident_zip': '11422',
  'incident_address': '236-17 129 AVENUE',
  'street_name': '129 AVENUE',
  'cross_street_1': '236 STREET',
  'cross_street_2': '237 STREET',
  'address_type': 'ADDRESS',
  'city': 'ROSEDALE',
  'status': 'Open',
  'resolution_description': 'If the abandoned vehicle meets the criteria to be classified as a derelict (i.e. junk) the Department of Sanitation (DSNY) will investigate and tag the vehicle within three business days.',
  'resolution_action_updated_date': '2023-12-05T12:00:00.000',
  'community_board': '13 QUEENS',
  'bbl': '4128770001',
  'borough': 'QUEENS',
  'x_coordinate_state_plane': '1058876',
  'y_coordinate_state_plane': '187277',
  'open_data_channel_type': 'PHONE',
  'park_facility_name': 'Unspecifie

Like the FEC, Socrata uses their own API to populate the tables when browsing data on sites powered by them.

**At-home exercise:** Try filtering a table on the [NYC Open Data Portal](https://data.cityofnewyork.us/), and find the API calls that makes.

In [9]:
pd.DataFrame(data)

,unique_key,created_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,...,borough,x_coordinate_state_plane,y_coordinate_state_plane,open_data_channel_type,park_facility_name,park_borough,latitude,longitude,location,facility_type
0,59632748,2023-12-05T12:00:00.000,DSNY,Department of Sanitation,Derelict Vehicles,Derelict Vehicles,Street,11422,236-17 129 AVENUE,129 AVENUE,...,QUEENS,1058876,187277,PHONE,Unspecified,QUEENS,40.680394296076656,-73.73094361326487,"{'latitude': '40.680394296076656', 'longitude'...",NaN
1,59632747,2023-12-05T12:00:00.000,DSNY,Department of Sanitation,Derelict Vehicles,Derelict Vehicles,Street,10018,555 TENTH AVENUE,TENTH AVENUE,...,MANHATTAN,985344,215745,PHONE,Unspecified,MANHATTAN,40.75884659915633,-73.99605105581034,"{'latitude': '40.75884659915633', 'longitude':...",NaN
2,59635445,2023-12-05T12:00:00.000,DSNY,Department of Sanitation,Derelict Vehicles,Derelict Vehicles,Street,11203,743 CLARKSON AVENUE,CLARKSON AVENUE,...,BROOKLYN,1002471,178427,PHONE,Unspecified,BROOKLYN,40.6563988449548,-73.93432978041173,"{'latitude': '40.6563988449548', 'longitude': ...",DSNY Garage
3,59635444,2023-12-05T12:00:00.000,DSNY,Department of Sanitation,Derelict Vehicles,Derelict Vehicles,Street,11213,736 CROWN STREET,CROWN STREET,...,BROOKLYN,1002797,181675,PHONE,Unspecified,BROOKLYN,40.66531320926436,-73.93314591173261,"{'latitude': '40.66531320926436', 'longitude':...",DSNY Garage
4,59632746,2023-12-05T12:00:00.000,DSNY,Department of Sanitation,Derelict Vehicles,Derelict Vehicles,Street,11203,585 SCHENECTADY AVENUE,SCHENECTADY AVENUE,...,BROOKLYN,1002531,179492,PHONE,Unspecified,BROOKLYN,40.65932191161629,-73.93411064761935,"{'latitude': '40.65932191161629', 'longitude':...",DSNY Garage
5,59635447,2023-12-05T12:00:00.000,DSNY,Department of Sanitation,Derelict Vehicles,Derelict Vehicles,Street,10463,3045 GODWIN TERRACE,GODWIN TERRACE,...,BRONX,1010169,259447,PHONE,Unspecified,BRONX,40.87875827837968,-73.9062730360959,"{'latitude': '40.87875827837968', 'longitude':...",NaN
6,59642268,2023-12-05T12:00:00.000,DSNY,Department of Sanitation,Derelict Vehicles,Derelict Vehicles,Street,11203,485 TROY AVENUE,TROY AVENUE,...,BROOKLYN,1001651,180442,PHONE,Unspecified,BROOKLYN,40.66193122462149,-73.93727993767622,"{'latitude': '40.66193122462149', 'longitude':...",DSNY Garage
7,59639609,2023-12-05T12:00:00.000,DSNY,Department of Sanitation,Derelict Vehicles,Derelict Vehicles,Street,11433,150-30 YATES ROAD,YATES ROAD,...,QUEENS,1039966,192095,PHONE,Unspecified,QUEENS,40.69375779116747,-73.7990815558646,"{'latitude': '40.69375779116747', 'longitude':...",NaN
8,59640911,2023-12-05T12:00:00.000,DSNY,Department of Sanitation,Derelict Vehicles,Derelict Vehicles,Street,11234,1578 EAST 51 STREET,EAST 51 STREET,...,BROOKLYN,1004733,165936,PHONE,Unspecified,BROOKLYN,40.6221087899582,-73.92621524071251,"{'latitude': '40.6221087899582', 'longitude': ...",NaN
9,59635446,2023-12-05T12:00:00.000,DSNY,Department of Sanitation,Derelict Vehicles,Derelict Vehicles,Street,11419,116-98 107 AVENUE,107 AVENUE,...,QUEENS,1032766,188473,PHONE,Unspecified,QUEENS,40.683858652592,-73.82507161090646,"{'latitude': '40.683858652592', 'longitude': '...",NaN


Coincidence there were exactly 1,000 results?

### Pagination

- Most APIs limit the number of results returned.
- [Socrata defaults to 1,000.](https://dev.socrata.com/docs/queries/limit.html)
- Need to use a loop with parameters like [`$limit`](https://dev.socrata.com/docs/queries/limit.html)+[`$offset`](https://dev.socrata.com/docs/queries/offset.html) (Socrata) or `page`+`per_page` ([FEC](https://api.open.fec.gov/developers/))
   - [`concat()`](https://pandas.pydata.org/pandas-docs/version/1.5/user_guide/merging.html#concatenating-objects) to DataFrame

## Things are going to differ by API

- Endpoints
- Supported parameters
- Response structure
   - [`json_normalize()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html) can help
- Quality of documentation
- Helpfulness of errors
- Size/helpfulness of community

Gotta read and experiment.

## [Final Project](https://python-public-policy.afeld.me/en/nyu/final_project.html)

- You should have received feedback on your proposal.
- Reminder that it's peer-graded.
   - You should see the notebooks you need to review come through Brightspace.
   - This is an opportunity to see how different people solve different problems.
   - You will lose points if you don't complete your peer grading.

## [Schedule](https://python-public-policy.afeld.me/en/nyu/syllabus.html#schedule)

## [Course evaluations](https://coursefeedback.nyu.edu/nyu/)

They are:

- Totally anonymous
- Not visible to me until grades are released
- A big help. Some things I took from the past:
    - Making assignments more rigorous
    - Students are hungry for more
    - People like the in-class exercises

Please complete now, if you haven't already, as they are due tonight.